In [1]:
%pip install tensorflow==2.13.0
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
%pip install pydub librosa soundfile tqdm keras==2.13.1
%pip install TTS==0.22.0

  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.14.1
    Uninstalling typing_extensions-4.14.1:
      Successfully uninstalled typing_extensions-4.14.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.31.0 requires typing_extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.
typeguard 4.4.4 requires typing_extensions>=4.14.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.7.1+cu126 requires typing-extensions>=4.10.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.6.3 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.16.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
optree 0.17.0 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.

[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.4 requires typing_extensions>=4.14.0, but you have typing-extensions 4.12.2 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.

[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: TTS==0.22.0 in c:\users\dingus\appdata\local\programs\python\python310\lib\site-packages (0.22.0)
  Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.14.1 which is incompatible.

[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# TensorFlow Lite Keyword Spotting Model Training
# Optimized for Embedded Hardware Deployment

# @title Train Model
import tensorflow as tf
import numpy as np
# tf.config.set_visible_devices([], 'GPU')
# Add this block of code
# Place this code block IMMEDIATELY after your imports
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Set memory growth to be True for all GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print(f"{len(gpus)} Physical GPUs, memory growth enabled.")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)





import tensorflow_model_optimization as tfmot
import os
import zipfile
import shutil
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import json
import random
from collections import defaultdict

print(f"\nSetup Complete!")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
tf
# @title Configuration Parameters
class Config:
    # Model Architecture Parameters
    MODEL_SIZE = "small"  # @param ["small", "medium", "large"]

    # Audio Processing Parameters
    SAMPLE_RATE = 16000
    DURATION = 1.0  # seconds
    N_MELS = 40
    N_FFT = 512
    HOP_LENGTH = 256

    # Training Parameters
    BATCH_SIZE = 4
    EPOCHS = 50
    LEARNING_RATE = 0.001
    VALIDATION_SPLIT = 0.2

    # Model Size Configurations
    MODEL_CONFIGS = {
        "small": {
            "conv_filters": [32, 64],
            "dense_units": 64,
            "dropout_rate": 0.3
        },
        "medium": {
            "conv_filters": [64, 128, 128],
            "dense_units": 128,
            "dropout_rate": 0.4
        },
        "large": {
            "conv_filters": [128, 256, 256, 512],
            "dense_units": 256,
            "dropout_rate": 0.5
        }
    }

    # Quantization Options
    QUANTIZE_MODEL = True  # @param {type:"boolean"}

    # Data Augmentation
    USE_DATA_AUGMENTATION = True  # @param {type:"boolean"}

config = Config()

# @title Locate and Extract Dataset from Local File
# This cell looks for 'speech_commands_dataset_local.zip' in the Colab root directory.
# If you haven't uploaded it, please do so using the file browser on the left.

# Define file and directory paths
zip_file_name = "speech_commands_dataset_local.zip"
dataset_path = "speech_commands_dataset"

# Check if the zip file exists in the current Colab environment
if not os.path.exists(zip_file_name):
    # If the file is not found, raise an error with instructions
    error_message = (
        f"ERROR: Dataset file '{zip_file_name}' not found.\n\n"
        "Please upload the dataset zip file to the Colab session's root directory.\n"
        "You can do this by clicking the 'Files' icon (folder symbol) on the left sidebar,\n"
        "and then clicking the 'Upload to session storage' button (page with an up arrow)."
    )
    raise FileNotFoundError(error_message)

print(f"Found local dataset: {zip_file_name}")

# Clean up any previous extraction to ensure a fresh start
if os.path.exists(dataset_path):
    print(f"Removing existing directory: {dataset_path} to ensure a clean extraction.")
    shutil.rmtree(dataset_path)

# Extract the dataset
print(f"Extracting {zip_file_name} to ./{dataset_path}/...")
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)
print("Extraction complete.")

# List available classes (excluding _background_noise_)
# This assumes the zip file extracts into subdirectories for each class.
try:
    classes = [d for d in os.listdir(dataset_path)
               if os.path.isdir(os.path.join(dataset_path, d))
               and d != '_background_noise_']
    classes.sort()
    if not classes:
        raise ValueError("No class subdirectories were found in the extracted dataset.")
    print(f"Found {len(classes)} classes: {classes}")
except FileNotFoundError:
    error_message = (
        f"Error: The directory '{dataset_path}' was not created after extraction.\n"
        f"Please ensure '{zip_file_name}' creates a '{dataset_path}' directory "
        "or contains the class folders at its root level."
    )
    raise RuntimeError(error_message)


# @title Data Loading and Preprocessing Functions

def load_audio_file(file_path, target_sr=16000, duration=1.0):
    """Load and preprocess audio file"""
    try:
        # Load audio file
        audio, sr = librosa.load(file_path, sr=target_sr, duration=duration)

        # Pad or trim to exact duration
        target_length = int(target_sr * duration)
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
        else:
            audio = audio[:target_length]

        return audio
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def extract_mel_spectrogram(audio, sr=16000, n_mels=40, n_fft=512, hop_length=256):
    """Extract mel spectrogram features"""
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
    )
    # Convert to log scale
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return log_mel_spec

def augment_audio(audio, sr=16000):
    """Apply data augmentation to audio"""
    augmented = []

    # Original audio
    augmented.append(audio)

    # Time shifting
    shift_max = int(sr * 0.1)  # 100ms
    shift = np.random.randint(-shift_max, shift_max)
    if shift > 0:
        audio_shifted = np.pad(audio, (shift, 0), mode='constant')[:-shift]
    else:
        audio_shifted = np.pad(audio, (0, -shift), mode='constant')[-shift:]
    augmented.append(audio_shifted)

    # Noise addition
    noise_factor = 0.005
    noise = np.random.randn(len(audio)) * noise_factor
    audio_noisy = audio + noise
    augmented.append(audio_noisy)

    return augmented

def load_file_lists(dataset_path):
    """Load testing and validation file lists if available"""
    testing_list = []
    validation_list = []

    # Construct full paths inside the extracted directory
    testing_file = os.path.join(dataset_path, 'testing_list.txt')
    validation_file = os.path.join(dataset_path, 'validation_list.txt')

    if os.path.exists(testing_file):
        with open(testing_file, 'r') as f:
            # Fix path separators to be universal
            testing_list = [line.strip().replace('\\', '/') for line in f.readlines()]
        print(f"Loaded {len(testing_list)} testing files from testing_list.txt")

    if os.path.exists(validation_file):
        with open(validation_file, 'r') as f:
            # Fix path separators to be universal
            validation_list = [line.strip().replace('\\', '/') for line in f.readlines()]
        print(f"Loaded {len(validation_list)} validation files from validation_list.txt")

    return testing_list, validation_list

# @title Dataset Preparation

def prepare_dataset(dataset_path, classes, config, testing_list=None, validation_list=None):
    """Prepare training, validation, and testing datasets"""

    all_files = []
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

    # Collect all audio files
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(class_path, file_name)
                    relative_path = os.path.join(class_name, file_name).replace('\\', '/')
                    all_files.append((file_path, class_to_idx[class_name], relative_path))

    print(f"Total files found: {len(all_files)}")

    # Split into train/val/test based on provided lists or randomly
    train_files = []
    val_files = []
    test_files = []

    if testing_list and validation_list:
        print("Using provided testing_list.txt and validation_list.txt for splits.")
        testing_set = set(testing_list)
        validation_set = set(validation_list)

        for file_path, label, rel_path in all_files:
            if rel_path in testing_set:
                test_files.append((file_path, label))
            elif rel_path in validation_set:
                val_files.append((file_path, label))
            else:
                train_files.append((file_path, label))
    else:
        print("Splitting data randomly (80% train, 10% val, 10% test).")
        random.shuffle(all_files)
        n_test = int(len(all_files) * 0.10)
        n_val = int(len(all_files) * 0.10)

        test_files = [(fp, label) for fp, label, _ in all_files[:n_test]]
        val_files = [(fp, label) for fp, label, _ in all_files[n_test:n_test + n_val]]
        train_files = [(fp, label) for fp, label, _ in all_files[n_test + n_val:]]

    print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

    return train_files, val_files, test_files, class_to_idx

# Load file lists and prepare dataset
testing_list, validation_list = load_file_lists(dataset_path)
train_files, val_files, test_files, class_to_idx = prepare_dataset(
    dataset_path, classes, config, testing_list, validation_list
)

# @title Feature Extraction and Data Loading

def create_dataset(file_list, config, augment=False):
    """Create dataset from file list"""
    X = []
    y = []

    for i, (file_path, label) in enumerate(file_list):
        if i % 500 == 0:
            print(f"Processing {i}/{len(file_list)} files...")

        audio = load_audio_file(file_path, config.SAMPLE_RATE, config.DURATION)
        if audio is None:
            continue

        audios_to_process = [audio]
        # Data augmentation for training set
        if augment and config.USE_DATA_AUGMENTATION:
            audios_to_process.extend(augment_audio(audio, config.SAMPLE_RATE)[1:]) # Skip original

        for proc_audio in audios_to_process:
            mel_spec = extract_mel_spectrogram(
                proc_audio, config.SAMPLE_RATE, config.N_MELS,
                config.N_FFT, config.HOP_LENGTH
            )
            X.append(mel_spec)
            y.append(label)

    X = np.array(X)
    y = np.array(y)

    # Add channel dimension for CNN
    X = np.expand_dims(X, axis=-1)

    print(f"Dataset shape: {X.shape}, Labels shape: {y.shape}")
    return X, y

# Create datasets
print("Creating training dataset...")
X_train, y_train = create_dataset(train_files, config, augment=True)

print("\nCreating validation dataset...")
X_val, y_val = create_dataset(val_files, config, augment=False)

print("\nCreating test dataset...")
X_test, y_test = create_dataset(test_files, config, augment=False)

# Normalize features
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Convert labels to categorical
num_classes = len(classes)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print(f"\nFinal shapes - Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# @title Model Architecture

def create_model(input_shape, num_classes, config):
    """Create CNN model for keyword spotting"""

    model_config = config.MODEL_CONFIGS[config.MODEL_SIZE]
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    # Add a rescaling layer to normalize input from [-80, 0] to [-1, 1]
    # This helps with quantization
    model.add(layers.Rescaling(1./80., offset=-0.5))

    # Convolutional blocks
    for i, filters in enumerate(model_config['conv_filters']):
        model.add(layers.Conv2D(filters, (3, 3), activation='relu', padding='same'))
        if i > 0: # No pooling after first conv layer
            model.add(layers.MaxPooling2D((2, 1)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.25))

    # Global average pooling to reduce parameters
    model.add(layers.GlobalAveragePooling2D())

    # Dense layers
    model.add(layers.Dense(model_config['dense_units'], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(model_config['dropout_rate']))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Create and compile model
input_shape = X_train.shape[1:]
model = create_model(input_shape, num_classes, config)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# @title Training

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
]

# Train the model
print("Starting training...")
history = model.fit(
    X_train, y_train,
    batch_size=config.BATCH_SIZE,
    epochs=config.EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

# Load the best model saved by ModelCheckpoint
model.load_weights('best_model.h5')

# @title Evaluation and Visualization

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

# Confusion Matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=classes))

# @title Model Optimization for Embedded Hardware

def quantize_model(model, X_train_sample):
    """Apply post-training quantization"""

    # Representative dataset for quantization
    def representative_data_gen():
        # Use a random subset of the training data
        for i in np.random.choice(X_train_sample.shape[0], 100):
            yield [X_train_sample[i:i+1].astype(np.float32)]

    # Convert to TensorFlow Lite with quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    quantized_tflite_model = converter.convert()
    return quantized_tflite_model

def create_tflite_model(model, quantize=False, X_train_sample=None):
    """Convert Keras model to TensorFlow Lite"""

    if quantize and X_train_sample is not None:
        print("Creating quantized TensorFlow Lite model...")
        tflite_model = quantize_model(model, X_train_sample)
    else:
        print("Creating standard (float32) TensorFlow Lite model...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()

    return tflite_model

# Create TensorFlow Lite models
# Standard TFLite model
tflite_model = create_tflite_model(model, quantize=False)

# Quantized TFLite model (if enabled)
if config.QUANTIZE_MODEL:
    tflite_quantized_model = create_tflite_model(model, quantize=True,
                                               X_train_sample=X_train)

# @title Model Size Comparison and Performance Testing

def get_model_size(model_content):
    """Get model size in KB"""
    return len(model_content) / 1024

def test_tflite_model(tflite_model_content, X_test_data, y_test_data):
    """Test TensorFlow Lite model accuracy"""

    interpreter = tf.lite.Interpreter(model_content=tflite_model_content)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    correct_predictions = 0
    num_test_samples = len(X_test_data)

    for i in range(num_test_samples):
        input_data = np.expand_dims(X_test_data[i], axis=0)

        # Check if model is quantized and scale input accordingly
        if input_details['dtype'] == np.int8:
            input_scale, input_zero_point = input_details['quantization']
            input_data = (input_data / input_scale + input_zero_point).astype(np.int8)

        interpreter.set_tensor(input_details['index'], input_data)
        interpreter.invoke()

        output_data = interpreter.get_tensor(output_details['index'])[0]

        # De-quantize output if necessary to compare
        if output_details['dtype'] == np.int8:
            output_scale, output_zero_point = output_details['quantization']
            output_data = (output_data.astype(np.float32) - output_zero_point) * output_scale

        predicted_class = np.argmax(output_data)
        true_class = np.argmax(y_test_data[i])

        if predicted_class == true_class:
            correct_predictions += 1

    accuracy = correct_predictions / num_test_samples
    return accuracy

# Compare model sizes and performance
print("Model Comparison:")
print("-" * 50)

# Original Keras model
model.save('/tmp/temp_model.h5')
keras_size = os.path.getsize('/tmp/temp_model.h5')
print(f"Keras Model Size: {keras_size / 1024:.2f} KB")
print(f"Keras Model Test Accuracy: {test_accuracy:.4f}")
print("-" * 50)

# TensorFlow Lite model
tflite_size = get_model_size(tflite_model)
tflite_accuracy = test_tflite_model(tflite_model, X_test, y_test)
print(f"TFLite (Float32) Model Size: {tflite_size:.2f} KB")
print(f"TFLite (Float32) Model Test Accuracy: {tflite_accuracy:.4f}")
print("-" * 50)


# Quantized model (if created)
if config.QUANTIZE_MODEL:
    tflite_quant_size = get_model_size(tflite_quantized_model)
    tflite_quant_accuracy = test_tflite_model(tflite_quantized_model, X_test, y_test)
    print(f"TFLite (INT8) Quantized Model Size: {tflite_quant_size:.2f} KB")
    print(f"TFLite (INT8) Quantized Model Test Accuracy: {tflite_quant_accuracy:.4f}")
    print(f"\nSize Reduction (Float32 -> INT8): {(1 - tflite_quant_size/tflite_size)*100:.1f}%")
    print("-" * 50)

# @title Save Models and Configuration

# Save Keras model
model.save('keyword_spotting_model.h5')
print("Saved Keras model: keyword_spotting_model.h5")

# Save TensorFlow Lite model
with open('keyword_spotting_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Saved TFLite (Float32) model: keyword_spotting_model.tflite")

# Save quantized model
if config.QUANTIZE_MODEL:
    with open('keyword_spotting_model_quantized.tflite', 'wb') as f:
        f.write(tflite_quantized_model)
    print("Saved TFLite (INT8) model: keyword_spotting_model_quantized.tflite")

# Save class labels and configuration
labels_info = {
    'classes': classes,
    'class_to_idx': class_to_idx,
    'idx_to_class': {v: k for k, v in class_to_idx.items()},
    'num_classes': num_classes
}

config_info = {
    'sample_rate': config.SAMPLE_RATE,
    'duration': config.DURATION,
    'n_mels': config.N_MELS,
    'n_fft': config.N_FFT,
    'hop_length': config.HOP_LENGTH,
    'model_size': config.MODEL_SIZE,
    'input_shape': list(input_shape),
    'quantized': config.QUANTIZE_MODEL
}

with open('labels.json', 'w') as f:
    json.dump(labels_info, f, indent=2)
print("Saved labels mapping: labels.json")

with open('config.json', 'w') as f:
    json.dump(config_info, f, indent=2)
print("Saved model configuration: config.json")


# @title Generate Deployment Files

# --- Create C++ header file for embedded deployment ---
def create_cpp_header(model_data, var_name="model_data"):
    """Create C++ header file with model data as a char array"""

    header_content = f"// Model data generated from TensorFlow Lite\n\n"
    header_content += f"const unsigned int {var_name}_len = {len(model_data)};\n"
    header_content += f"const unsigned char {var_name}[] __attribute__((aligned(16))) = {{"

    # Add model data as hex values
    for i, byte in enumerate(model_data):
        if i % 12 == 0:
            header_content += "\n  "
        header_content += f"0x{byte:02x},"

    header_content = header_content.rstrip(',')
    header_content += "\n};"
    return header_content

# Create header files
tflite_header = create_cpp_header(tflite_model, var_name="g_model_float_data")
with open('model_float.h', 'w') as f:
    f.write(tflite_header)
print("Generated C++ header for float model: model_float.h")

if config.QUANTIZE_MODEL:
    tflite_quant_header = create_cpp_header(tflite_quantized_model, var_name="g_model_quant_data")
    with open('model_quantized.h', 'w') as f:
        f.write(tflite_quant_header)
    print("Generated C++ header for quantized model: model_quantized.h")


# @title Create and Download Deployment Package

# Zip all the necessary files for deployment
deployment_files = [
    'keyword_spotting_model.h5',
    'keyword_spotting_model.tflite',
    'config.json',
    'labels.json',
    'model_float.h',
    'arduino_keyword_spotting.ino',
    'python_inference.py'
]
if config.QUANTIZE_MODEL:
    deployment_files.append('keyword_spotting_model_quantized.tflite')
    deployment_files.append('model_quantized.h')

zip_filename = 'deployment_package.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in deployment_files:
        if os.path.exists(file):
            zipf.write(file)
            print(f"Added {file} to zip.")
        else:
            print(f"Warning: {file} not found and was not added to the zip.")

# Provide the zip file for download
print(f"\nCreated {zip_filename}. Click the folder icon on the left to find it, then right-click to download.")
print("Alternatively, run the cell below to trigger an automatic download.")

# @title Download the Package
from google.colab import files
files.download(zip_filename)

# @title Next Steps
print("""
=================================
Deployment Package Contents
=================================
Your `deployment_package.zip` contains:
  - *.h5: The full Keras model for further training or fine-tuning.
  - *.tflite: The converted TensorFlow Lite models (float and/or int8).
  - *.h: The TFLite models converted into C++ header files for embedding.
  - config.json: The audio preprocessing parameters.
  - labels.json: The mapping from class index to class name.
  - arduino_keyword_spotting.ino: An example sketch for Arduino/ESP32.
  - python_inference.py: A script to test the TFLite models on your computer.

=================================
How to Use the Files
=================================
1. Local Testing:
   - Unzip the package.
   - Run the Python script with a WAV file to test inference:
     `python3 python_inference.py --model keyword_spotting_model_quantized.tflite --file your_audio.wav`

2. Embedded Deployment (e.g., ESP32):
   - Open the Arduino IDE and set up your board.
   - Install the necessary TFLite for Microcontrollers library.
   - Create a new sketch and copy the contents of `arduino_keyword_spotting.ino`.
   - In the same sketch folder, add the C++ model header you want to use (`model_quantized.h` is recommended).
   - IMPORTANT: You must implement the audio capture and feature extraction pipeline on the device. This notebook provides the model, but the on-device audio processing is a separate, significant task.
""")

ModuleNotFoundError: No module named 'seaborn'